# 爬虫初步实战：从虾皮网获取商品信息

### 先下载selenium

In [20]:
conda install -c conda-forge selenium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/shengkailin/opt/anaconda3/envs/keras

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.11.8  |       h033912b_0         145 KB  conda-forge
    certifi-2020.6.20          |   py37h2987424_2         151 KB  conda-forge
    openssl-1.1.1h             |       haf1e3a3_0         1.9 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    selenium-3.141.0           |py37h60d8a13_1002         865 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 

In [30]:
#through selenium get the cookie and then from the cookie get the x-csrftoken
from selenium import webdriver
#must download the chromedriver
driver=webdriver.Chrome('/Users/shengkailin/opt/chromdrivers/chromedriver')
driver.get('https://shopee.tw/flash_sale?promotionId=2004620383')

In [32]:
#get the cookie and csrftoken
ary=[]
csrftoken=''
for ele in driver.get_cookies():
    if ele.get('name')=='csrftoken':
        csrftoken=ele.get('value')
    ary.append('{}={}'.format(ele.get('name'),ele.get('value')))
cookie=';'.join(ary)    

In [33]:
csrftoken

'Q1bMYcfcys12XJqWhpCclf9BceRYcUZ6'

In [34]:
cookie

'_gid=GA1.2.937594521.1605116215;_ga=GA1.2.707160462.1605116215;AMP_TOKEN=%24NOT_FOUND;SPC_T_ID="Gs0dJlP4dz6uJ2YcflduBhbT0Kw8KP/X+Z9Hhhba71RtpNjYI7/Pm/IAVmUlqqVx8Zw7CZQqexOLbNveMo2AbwpFqlFe0VQ9gmLp8TuZBYQ=";SPC_R_T_IV="0dwlZ8OV23qh37sz1DDkNQ==";SPC_R_T_ID="Gs0dJlP4dz6uJ2YcflduBhbT0Kw8KP/X+Z9Hhhba71RtpNjYI7/Pm/IAVmUlqqVx8Zw7CZQqexOLbNveMo2AbwpFqlFe0VQ9gmLp8TuZBYQ=";SPC_SI=mall.gZkozygixGZdcG8NOR2MxhwlMMRinJT3;csrftoken=Q1bMYcfcys12XJqWhpCclf9BceRYcUZ6;SPC_U=-;_fbp=fb.1.1605116212185.961408901;SPC_F=a8GDg7hqnO8zLFhZC8Up5GvndmJGELLJ;REC_T_ID=7b534d98-2444-11eb-9879-b4969130c7a6;SPC_T_IV="0dwlZ8OV23qh37sz1DDkNQ==";SPC_EC=-;SPC_IA=-1;_gcl_au=1.1.672573601.1605116211'

In [36]:
import requests 
import json
#through seme
#some headers are really important such as x-csrftoken,referer,cookie
headers={
    
    'x-csrftoken': csrftoken,
    'referer': 'https://shopee.tw/flash_sale?promotionId=2004620383',
    #'content-type': 'application/json',
    #'if-none-match-': '55b03-2f8cd550ac0b38f119d2078b84c40c2b',
    #'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    #'user-agent': 'Mozilla/5.0',
    #'x-requested-with': 'XMLHttpRequest',
    #'x-shopee-language': 'zh-Hant',
    'cookie':cookie
}
payload=json.loads('''{"promotionid":2004620383,"categoryid":0,"itemids":[3550408258,7644201748,3625512084,6036230825,7961404634,2436919083,617540013,6842639345,1464315295,3737140201,3357269576,7843990457],"sort_soldout":false,"limit":12,"need_personalize":true,"with_dp_items":true}''')
#把在Headers里面找到的url放到这里，给改地址发送请求，返回的数据存储在json格式中
res=requests.post('https://shopee.tw/api/v2/flash_sale/flash_sale_batch_get_items',json=payload,headers=headers)


In [38]:
#empty result means our srawler has been blocked out
res.json()

{'version': 'ae325a2fc6c62c4313b0ec93f9d634ba',
 'data': {'items': [{'itemid': 3550408258,
    'brand_sale_brand_custom_logo': None,
    'image': '4a15e92767bcd4b45c191c81ee7369c6',
    'shopid': 261978965,
    'voucher': {'min_spend': 19900000,
     'discount_percentage': 0,
     'coin_percentage': None,
     'discount_value': 2500000,
     'voucher_code': 'ZCX60013',
     'reward_type': 0,
     'promotionid': 62030873},
    'raw_discount': 59,
    'price_before_discount': 13800000,
    'flash_sale_type': 0,
    'promo_overlay_image': 'a96d51fe34012632de5d3eb5f3ca076c',
    'modelids': [30852899928,
     30852899934,
     30852899937,
     30852899945,
     30852899930,
     30852899938,
     30852899940,
     30852899941,
     30852899944,
     50852884840,
     30852899931,
     30852899935,
     30852899942,
     30852899927,
     30852899929,
     30852899932,
     30852899933,
     30852899936,
     30852899939,
     30852899943],
    'promo_images': ['80d6ac4bf9fced46ade97800cf5

In [39]:
#scraw all itemid through an api
import requests
res=requests.get('https://shopee.tw/api/v2/flash_sale/get_all_itemids?need_personalize=true&promotionid=2004620383&sort_soldout=false')
res.json()

{'version': '7696ddbf8c1cf9e08dae7af35a673ffb',
 'data': {'selling_out_item_brief_list': [],
  'items': [],
  'mega_sale_items': [],
  'item_brief_list': [{'itemid': 3550408258,
    'recommendation_info': 'QUE:FSRANK4,SLT:justforyoufs_slot_00,PID:11,TFS:justforyoufs_slot_00_TW,SEC:justforyoufs_sec_00,BND:justforyoufs,EPT:justforyoufs',
    'from': None,
    'catid': 48,
    'cat_label': 0,
    'is_soldout': False},
   {'itemid': 7644201748,
    'recommendation_info': 'QUE:FSRANK4,SLT:justforyoufs_slot_00,PID:11,TFS:justforyoufs_slot_00_TW,SEC:justforyoufs_sec_00,BND:justforyoufs,EPT:justforyoufs',
    'from': None,
    'catid': 10,
    'cat_label': 0,
    'is_soldout': False},
   {'itemid': 3625512084,
    'recommendation_info': 'QUE:FSRANK4,SLT:justforyoufs_slot_00,PID:11,TFS:justforyoufs_slot_00_TW,SEC:justforyoufs_sec_00,BND:justforyoufs,EPT:justforyoufs',
    'from': None,
    'catid': 2,
    'cat_label': 0,
    'is_soldout': False},
   {'itemid': 6036230825,
    'recommendation_in